In [1]:
import os
import subprocess
import multiprocessing 
import threading
from functools import partial
from PIL import Image

In [2]:
def makedir(dirName):
    cmd="mkdir {}"
    os.system(cmd.format( dirName))

In [3]:
thCount=multiprocessing.cpu_count()
thCount

8

In [4]:
#To further improve efficiency of the code, make the process detachable
#to make this function detachable, remove all the use of global variable, i.e. new/oldDirName, folder
#possibly by making them input parameters
def linkImg(ID, directoryList):
    
    print(directoryList[ID]+' ')
    #create a new subfolder in new directory
    makedir(os.path.join(longPath, newDirName, folder, '"'+directoryList[ID]+'"'))
    #in the subfolder of old directory
    fullPath=os.path.join(longPath, oldDirName, folder, directoryList[ID])
    elementList=os.listdir(fullPath)
    if('.DS_Store' in elementList):
        elementList.remove('.DS_Store')
    
    #the following parameter is prepared seperate clip
    #timeMark for labeling frame No
    timeMark='';
    elementList.sort()
    iterator=0
    nameOfClip='clip{}'
    seperateClip=True
    lncmd="ln -sf {} {}"
    for element in elementList:
        if(int(element[6:10])%frameGap!=0):
            continue
        #check subject number, if doesn't exist, create new subject folder under new directory
        subjectNo=element[:2]
        if(not os.path.exists(os.path.join(longPath, newDirName, folder, directoryList[ID], subjectNo))):
            #print('creating folder')
            timeMark='';
            iterator=0
            makedir(os.path.join(longPath, newDirName, folder, '"'+directoryList[ID]+'"', subjectNo))
        
        
        
        #determine if need to seperate clips from clips
        if(seperateClip):
            #if is start of series
            if timeMark == '':
                NOP=nameOfClip.format(iterator)
                makedir(os.path.join(longPath, newDirName, folder, '"'+directoryList[ID]+'"', subjectNo, NOP))

            else:
                #if gap exist
                if(int(element[6:10])-int(timeMark)>2*frameGap):
                    iterator+=1
                    NOP=nameOfClip.format(iterator)
                    makedir(os.path.join(longPath, newDirName, folder, '"'+directoryList[ID]+'"', subjectNo, NOP))
                #else
                #link original to destination
                os.system(lncmd.format( os.path.join(longPath, oldDirName, folder, '"'+directoryList[ID]+'"', element), 
                        os.path.join(longPath, newDirName, folder, '"'+directoryList[ID]+'"', subjectNo, NOP, element)))
            timeMark=element[6:10]
        else:        
            #make softlink for the element
        
            os.system(lncmd.format( os.path.join(longPath, oldDirName, folder, '"'+directoryList[ID]+'"', element), 
                        os.path.join(longPath, newDirName, folder, '"'+directoryList[ID]+'"', subjectNo, element)))

In [5]:
newDirName='VideoClasBinned'
oldDirName='VideoClas'
longPath='/home/billy/Desktop'
cmd="mkdir {}"
ignoreList=['CLAP','INCISION','INTUBATION-STYLETE','MEDICATION','PLACE CUFF']
tinyList=['BAGGING','CHEST-TUBE PREP','KING AIRWAY','SPLINTING','WRAP HEAD WOUND']
smallList=['BLOOD-PRESSURE CUFF','COMBAT GAUZE','COMBAT TOURNIQUET','ECG LEADS','INTUBATION','SUTURING','VITAL CHECKING']
mediumList=['CHEST-TUBE','IM ADMINISTRATION','IV TOURNIQUET']
frameGap=10
maxSampleCount=35
makedir(os.path.join(longPath,newDirName))
#symbol link from the original file to new location after classification
#symbol link example: ln -sf source destination
folderList=['Training','Validation','Test']

for folder in folderList:
    print('Performing operation for '+ folder+'\n')
    if os.path.exists(os.path.join(longPath, newDirName, folder)):
        print('Folder '+ folder +' already exists, skip\n')
        continue
    makedir(os.path.join(longPath, newDirName, folder))
    target="{}/{}".format(oldDirName, folder)
    dirList=os.listdir(os.path.join(longPath, target))
    if('.DS_Store' in dirList):
        dirList.remove('.DS_Store')
    for cate in dirList:
        if cate in ignoreList:
            dirList.remove(cate)
    thCount=multiprocessing.cpu_count()
    pool = multiprocessing.Pool(thCount)
    partial_linkImg=partial(linkImg, directoryList=dirList)
    N = len(dirList)
    _=pool.map(partial_linkImg,range(N))
    pool.close()
    pool.join()

Performing operation for Training

CHEST-TUBE 
KING AIRWAY 
DRAW MEDICATION 
COMBAT TOURNIQUET 
CPR (COMPRESSION) 
INTUBATION 
COMBAT GAUZE 
IO LINE 
BLOOD-PRESSURE CUFF 
SPLINTING 
ECG LEADS 
IV TOURNIQUET 
VITAL CHECKING 
SUTURING 
PULSE-OX 
ADMINISTER MEDICATION 
IV LINE 
SWAB AREA WITH ALCOHOL 
ORAL AIRWAY 
CPR (BREATH) 
WRAP HEAD WOUND 
CHEST-TUBE PREP 
BAGGING 
IM ADMINISTRATION 
Performing operation for Validation

COMBAT GAUZE 
DRAW MEDICATION 
COMBAT TOURNIQUET 
KING AIRWAY 
BLOOD-PRESSURE CUFF 
INTUBATION 
IO LINE 
CHEST-TUBE 
SPLINTING 
ECG LEADS 
IV TOURNIQUET 
VITAL CHECKING 
SUTURING 
PULSE-OX 
ADMINISTER MEDICATION 
IV LINE 
SWAB AREA WITH ALCOHOL 
ORAL AIRWAY 
WRAP HEAD WOUND 
BAGGING 
IM ADMINISTRATION 
Performing operation for Test

KING AIRWAY 
CHEST-TUBE 
COMBAT GAUZE 
DRAW MEDICATION 
CPR (COMPRESSION) 
COMBAT TOURNIQUET 
IO LINE 
INTUBATION 
BLOOD-PRESSURE CUFF 
SPLINTING 
ECG LEADS 
IV TOURNIQUET 
VITAL CHECKING 
SUTURING 
PULSE-OX 
ADMINISTER MEDICATION 
IV LINE

In [ ]:
#old code

In [4]:
newDirName='ResizeBinned'
oldDirName='Resize'
cmd="mkdir {}"
makedir(newDirName)
#symbol link from the original file to new location after classification
#symbol link example: ln -sf source destination
folderList=['Test','Training' , 'Validation']
longPath='/Users/lingfengli/Desktop/SIR/cela-tf-starter'
for folder in folderList:
    print('Performing operation for '+ folder+'\n')
    makedir(os.path.join(longPath, newDirName, folder))
    target="{}/{}".format(oldDirName, folder)
    dirList=os.listdir(os.path.join(longPath, target))
    if('.DS_Store' in dirList):
        dirList.remove('.DS_Store')
    #a processed dirList escaping space character
    dirListP=[]
#     for folderName in dirList:
#         folderName=folderName.replace(" ", "\" \"")
#         dirListP.append(folderName)
#     for subfolder in dirListP:
        
    for subfolder in dirList:
        print(subfolder+' ')
        #create a new subfolder in new directory
        makedir(os.path.join(longPath, newDirName, folder, '"'+subfolder+'"'))
        #in the subfolder of old directory
        fullPath=os.path.join(longPath, oldDirName, folder, subfolder)
        elementList=os.listdir(fullPath)
        if('.DS_Store' in elementList):
            elementList.remove('.DS_Store')
        for element in elementList:
            #check subject number, if doesn't exist, create new subject folder under new directory
            subjectNo=element[:2]
            if(not os.path.exists(os.path.join(longPath, newDirName, folder, subfolder, subjectNo))):
                #print('creating folder')
                makedir(os.path.join(longPath, newDirName, folder, '"'+subfolder+'"', subjectNo))
            #make softlink for the element
            lncmd="ln -sf {} {}"
            os.system(lncmd.format( os.path.join(longPath, oldDirName, folder, '"'+subfolder+'"', element), 
                           os.path.join(longPath, newDirName, folder, '"'+subfolder+'"', subjectNo, element)))

Performing operation for Test

CPR (BREATH) 


In [83]:
folderList=['Test','Training' , 'Validation']
longPath='/Users/lingfengli/Desktop/SIR/cela-tf-starter'
# for folder in folderList:
#     target="/{}/{}".format(oldDirName, folder)
#     print(target)
os.system(cmd.format( os.path.join(longPath, newDirName, folderList[0])))
target="{}/{}".format(oldDirName, folderList[0])
print(target)


Resize/Test


In [115]:
dirList=os.listdir(os.path.join(longPath, target))
if('.DS_Store' in dirList):
    dirList.remove('.DS_Store')
#a processed dirList escaping space character
dirListP=[]
for folderName in dirList:
    folderName=folderName.replace(" ", "\" \"")
    dirListP.append(folderName)
for subfolder in dirListP:
    #create a new subfolder in new directory
    os.system(cmd.format( os.path.join(longPath, newDirName, folderList[0], subfolder)))
# for subfolder in dirList:    
#     #in the subfolder of old directory
#     fullPath=os.path.join(longPath, oldDirName, folderList[0], subfolder)
#     elementList=os.listdir(fullPath)



In [121]:
fullPath=os.path.join(longPath, oldDirName, folderList[0], dirList[0])
elementList=os.listdir(fullPath)
if('.DS_Store' in elementList):
    elementList.remove('.DS_Store')

In [134]:
for element in elementList:
    #check subject number, if doesn't exist, create new subject folder under new directory
    subjectNo=element[0:2]
    if(not os.path.exists(os.path.join(longPath, newDirName, folderList[0], dirList[0], subjectNo))):
        print('creating folder')
        makedir(os.path.join(longPath, newDirName, folderList[0], dirList[0], subjectNo))
    #make softlink for the element
    lncmd="ln -sf {} {}"
    os.system(lncmd.format( os.path.join(longPath, oldDirName, folderList[0], dirList[0], element), 
                           os.path.join(longPath, newDirName, folderList[0], dirList[0], subjectNo, element)))

In [10]:
a='123 123'

In [13]:
a.replace(" ", '" "')

'123" "123'

In [5]:
123%10

3